# Иерархическая индексация

До сих пор мы рассматривали главным образом одномерные и двумерные данные, которые сохраняются в объектах Pandas `Series` и `DataFrame` соответственно. Часта является полезным выйти за пределы таких размерностей и сохранять данные более высоких размерностей, такие, которые индексируются более чем одним или двумя ключами. В то время как Pandas предлагает объекты `Panel` и `Panel4D` которые нативным образом обрабатывают трехмерныей и четырехмерные данные ([Aside: Panel Data](https://jakevdp.github.io/PythonDataScienceHandbook/03.05-hierarchical-indexing.html#Aside:-Panel-Data)), на практике намного более частым шаблоном является применение _иерархического индексирования_ (также известного как _multi-indexing_) для включения множественных уровней индекса внутрь другого единственного индекса. Таким образом, данные высоких размерностей могут быть компактно представлены с помощью уже знакомых объектов: одномерных `Series` и двумерных `DataFrame`.

В этом разделе мы рвссмотрим прямое создание объектов `MultiIndex`, соображения при индексации, срезы и подсчет статистики для многоиндексных данных, и полезные функции для конвертирвания между простой и иерархической индексацией.

In [3]:
import pandas as pd
import numpy as np

## Мультииндексированный объект `Series`

Давайте начнём с представления двумерных данные с помощью одномерного объекта `Series`. Для правильности, мы предположим, это набор данных, в которых каждая точка имеет символьный и числовой ключ.

### Плохой путь

Допустим вы хотели бы отследить данные по штатам по двум разным годам. Используя инструменты Pandas, которые мы уже рассмотрели, вы могли бы решить, что проще использовать кортежи Python в качестве ключей:

In [4]:
index = [('California', 2000), ('California', 2010),
         ('New York', 2000), ('New York', 2010),
         ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561]
pop = pd.Series(populations, index=index)
pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

В таком случае вы можете напрямую индексировать или срезать данные с помощью мультииндекса:

In [5]:
pop[('California', 2010):('Texas', 2000)]

(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
dtype: int64

Но удобства здесь и заканчиваются. Например, если вам нужно извлечь все данные за 2010 год, вам понадобится достаточно запутанный (и потенциально медленный) подход:

In [6]:
pop[[i for i in pop.index if i[1] == 2010]]

(California, 2010)    37253956
(New York, 2010)      19378102
(Texas, 2010)         25145561
dtype: int64

Такой подход работает, но, в отличие от срезов, которые мы уже успели полюбить в Pandas, он недостаточно чистый (и недостаточно эффективный на больших наборах данных).

### Более правильный способ: Pandas `MultiIndex`

К счастью Pandas предлагает лучший способ. Наша индексация на основе кортежей, по сути является рудиментарным мультииндексом, и Pandas объект `MultiIndex` предлагает нам возможности, которые мы хотели бы получить. Мы можем создать мультииндекс из кортежей следующим образом:

In [9]:
index = pd.MultiIndex.from_tuples(index)
index

MultiIndex([('California', 2000),
            ('California', 2010),
            (  'New York', 2000),
            (  'New York', 2010),
            (     'Texas', 2000),
            (     'Texas', 2010)],
           )

Если мы переиндексируем нашу серию данных с помощью `MultiIndex`, то мы увидим иерархическое представление данных:

In [10]:
pop = pop.reindex(index)
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

Здесь первые две колонки объекта `Series` представляют мультииндексные значения, третья колонка представляет данные. Обратите внимание, что некоторые данные отсутствуют в первой колонке: в мультииндексном представлении любое пропущенное значение означает, что в реальности здесь находится такое же значение как и в предыдущей линии.

Теперь, для того, чтобы получить доступ ко всем данным, у которых второй индекс `2010`, мы можем использовать простой срез:

In [11]:
pop[:, 2010]

California    37253956
New York      19378102
Texas         25145561
dtype: int64

Результатом является одно-индексный массив только с теми ключами, в которых мы заинтересованы. Такой синтакс гораздо более удобный (и сама операция гораздо более эффективна) чем подход с кортежами с которого мы начинали.

### `MultiIndex` в качестве дополнительной размерности (измерения)

Возможно вы заметили, что мы могли бы легко сохранить данные в простом объекте `DataFrame` с индексом и метками колонок. В действительности, Pandas построен с учётом эквивалентности. Метод `unstack()` легко конвертирует мультииндексный объект `Series` в традиционно индексный объект `DataFrame`:

In [13]:
pop_df = pop.unstack()
pop_df

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


Метод `stack()` проводит обратную операцию:

In [14]:
pop_df.stack()

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

Учитывая вышеизложенное, вы могли бы задаться вопросом: зачем тогда нам нужна иерархическая индексация? Причина проста: таким же образом, каким мы использовали мульти-индексацию для представления двумерных данных в одномерном `Series`, мы также можем использовать мульти-индексацию для представления трех- и более высокоразрядных данных в объектах `Series` или `DataFrame`. Каждый дополнительные уровень в мульти-индексе представляет дополнительную размерность данных. Например, с `MultiIndex` мы можем добавить новую колонку демографических данных к каждому штату за каждый год (например, численность населения до 18 лет):

In [15]:
pop_df = pd.DataFrame({'total': pop,
                       'under18': [9267089, 9284094,
                                   4687374, 4318033,
                                   5906301, 6879014]})
pop_df

total  under18
California 2000  33871648  9267089
           2010  37253956  9284094
New York   2000  18976457  4687374
           2010  19378102  4318033
Texas      2000  20851820  5906301
           2010  25145561  6879014

Помимо этого, все универсальные функции (ufuncs) и любая другая функциональность, которую мы обсуждали в разделе [Operating on Data in Pandas](https://jakevdp.github.io/PythonDataScienceHandbook/03.03-operations-in-pandas.html), работает также и с иерархической индекскацией. Далее мы вычисляем долю людей до 18 лет по каждому году:

In [19]:
f_u18 = pop_df['under18'] / pop_df['total']
f_u18.unstack()

,2000,2010
California,0.273594,0.249211
New York,0.247010,0.222831
Texas,0.283251,0.273568


## Способы создания `MultiIndex`

Наиболее простой способ создания мульти-индексов для `Series` или `DataFrame` - просто передать в конструктов список двух или более массивов:

In [25]:
df = pd.DataFrame(
    np.random.rand(4, 2),
    index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
    columns=['data1', 'data2']
)
df

data1     data2
a 1  0.912457  0.443127
  2  0.909954  0.102083
b 1  0.916532  0.031081
  2  0.703099  0.911570

Аналогично, если вы передадите словарь с соответствующими ключами, Pandas автоматически определить это и использует `MultiIndex` по умолчанию:

In [27]:
data = {
    ('California', 2000): 33871648,
    ('California', 2010): 37253956,
    ('Texas', 2000): 20851820,
    ('Texas', 2010): 25145561,
    ('New York', 2000): 18976457,
    ('New York', 2010): 19378102
}
pd.Series(data)

California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
New York    2000    18976457
            2010    19378102
dtype: int64

### Явное создание `MultiIndex`

Для большей гибкости в том, как создается индекс, вы можете использовать конструкторы класса доступные в `pd.MultiIndex`. Например, как мы уже делали раньше, вы можете создать `MultiIndex` просто из списка массивов:

In [29]:
pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1, 2, 1, 2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [31]:
# Создание из списка кортежей
pd.MultiIndex.from_tuples([('a', 1), ('a', 2), ('b', 1), ('b', 2)])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [33]:
# Создание индекса из набора списков путем декартова произведения
pd.MultiIndex.from_product([['a', 'b'], [1, 2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

Аналогичным образом, вы можете создать `MultiIndex` напрямую, использую его внутреннее представление, путём передачи `levels` (список списков содержащих доступные значения индексов для каждого уровня) и `labels` (список списков меток):

In [35]:
pd.MultiIndex(
    levels=[['a', 'b'], [1, 2]],
    labels=[[0, 0, 1, 1], [0, 1, 0, 1]]
)

TypeError: MultiIndex.__new__() got an unexpected keyword argument 'labels'

https://jakevdp.github.io/PythonDataScienceHandbook/03.05-hierarchical-indexing.html#MultiIndex-level-names